In [ ]:
import os
import csv
import pandas as pd
import json
from tqdm import tqdm
import time



## Opportunity Detection Procedure

Opportunity detection was performed using a single analysis pipeline.
The same notebook was reused across tutor moves by substituting the
corresponding opportunity prompt for each construct.

For reproducibility, opportunity prompts were not executed jointly.
Instead, each tutor move was evaluated separately by loading its
construct-specific opportunity prompt and running the notebook end to
end. This ensures that opportunity identification remains aligned with
the theoretical definition of each tutor move.

To reproduce this process, users should select the desired opportunity
prompt from the prompts directory, update the prompt path in the
configuration cell below, and rerun the notebook for that construct.


In [ ]:
#If you are working from colab notebook, set your key in 'secrets' set name to 'gemini_key'
import google.generativeai as genai
from google.colab import userdata
userdata.get('gemini_key')
API_KEY = userdata.get('gemini_key')
genai.configure(api_key=API_KEY)
model = genai.GenerativeModel("gemini-2.5-pro")

In [ ]:
"""
REPLACE WITH AN OPPORTUNITY PROMPT from prompts/LLM Implementation of Tutors Moves in Transcripts Scoring Prompts
eg: REACT_ERRORS Opportunity Prompt
"""
def build_prompt(transcript_text: str) -> str:
    return f"""
You are an expert tutor reviewer.


Your task is to determine whether the following tutoring transcript contains a clear OPPORTUNITY for a tutor to REACT TO A STUDENT'S MATH ERROR.
---

An OPPORTUNITY TO REACT TO A STUDENT'S MATH ERROR exists when:
- A student makes a mistake while solving a math problem(ex: The student incorrectly solves 118 + 18 = 126)

Scoring rules:
-1: There was an opportunity for a tutor to react to a student's math error
-0: There was no opportunity for a tutor to react to a student's math error

If the student and the tutor did not work on any math problems then there was no opportunity.

---
Constraints:
- The transcript is not diarized. You'll need to infer who is speaking based on the text itself.
- Focus only on students making math errors, not whether the tutor reacted effectively or ineffectively.

---


Format your output in valid JSON using this format ONLY:


```json
{{
  "had_opportunity_to_react_to_errors": 1,
  "evidence": "at [00:09], The student made a conceptual error, defining 'mean' as 'The one that comes up the most."
}}


Audio transcript:
\"\"\"
{transcript_text}
\"\"\"
"""



In [ ]:
ROOT_DIR = ""
OUTPUT_CSV = ""
BATCH_SIZE = 20
WAIT_TIME_BETWEEN_BATCHES = 5 #To prevent Gemini from timing-out

fieldnames = [
    "file",
    "had_opportunity_to_react_to_errors",
    "evidence"
]

# Track already processed files
completed_files = set()
if os.path.exists(OUTPUT_CSV):
    df_existing = pd.read_csv(OUTPUT_CSV)
    completed_files = set(df_existing["file"].tolist())

# Collect new .vtt files
all_vtt_files = [
    f for f in os.listdir(ROOT_DIR)
    if f.endswith(".vtt") and f not in completed_files
]

def process_file(filename):
    file_path = os.path.join(ROOT_DIR, filename)
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            transcript_text = f.read()

        prompt = build_prompt(transcript_text)
        response = model.generate_content(prompt)
        content = response.text.strip()

        if content.startswith("```json"):
            content = content.replace("```json", "").replace("```", "").strip()

        scores = json.loads(content)

    #    model output with evidence
        print(f"\n📘 {filename} \n{json.dumps(scores, indent=2)}\n")

        
        row = {"file": filename}
        for dim in fieldnames[1:]:
            row[dim] = scores.get(dim, 0)

        return row

    except Exception as e:
        print(f"Error processing {filename}: {e}")
        return None

# batch processing
for batch_start in range(0, len(all_vtt_files), BATCH_SIZE):
    batch_files = all_vtt_files[batch_start: batch_start + BATCH_SIZE]
    print(f"\n Processing batch {batch_start // BATCH_SIZE + 1} ({len(batch_files)} files)")

    batch_results = []
    for filename in tqdm(batch_files):
        result = process_file(filename)
        if result:
            batch_results.append(result)

    # Save just the scores
    if batch_results:
        pd.DataFrame(batch_results).to_csv(
            OUTPUT_CSV, mode="a", header=not os.path.exists(OUTPUT_CSV), index=False
        )
        print(f"🍀Batch saved: {len(batch_results)} results")
    else:
        print("No results in this batch (all failed?)")

    print(f"Waiting {WAIT_TIME_BETWEEN_BATCHES} seconds before next batch...")
    time.sleep(WAIT_TIME_BETWEEN_BATCHES)
